In [ ]:
!python3 -m pip install PyMySQL
!python3 -m pip install SQLAlchemy
!python3 -m pip install google-cloud-storage
!python3 -m pip install --upgrade --quiet scikit-sound
!python3 -m pip install --upgrade --quiet pygame
!sudo apt-get -y install ffmpeg
!sudo apt-get -y install python3-pymysql

In [1]:
import sqlalchemy
import numpy as np

from google.cloud import storage
from numpy.fft import fft, ifft
from sksound.sounds import Sound

from tensorflow.keras.models import load_model

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
highpass=25

class StreamSpectrogram:
    
    def __init__(self, filename, win=32):
        sound = Sound(filename)        
        self.batch = sound.rate * 60
        self.data  = sound.data
        if len(self.data.shape) > 1:
            self.data = self.data[:, 0]    
        self.win = win
        
    def __iter__(self):
        self.i = 0
        return self

    def __next__(self):
        if (self.i + 1) * self.batch < len(self.data): 
            start = self.i       * self.batch
            stop  = (self.i + 1) * self.batch
            raw   = self.data[start:stop]
            spec  = fwd_spectrogram(raw, win=512 + 2 * highpass)[:, 0:256] 
            t,d   = spec.shape
            current = []        
            for i in range(self.win, t, self.win // 2):
                x      = np.reshape(spec[i - self.win:i], (self.win, d, 1))
                mu     = np.mean(x)
                std    = np.std(x) + 1.0
                window = (x - mu) / std
                current.append(window)
            self.i += 1
            return np.stack(current)
        else:
            raise StopIteration
        
def fwd_spectrogram(audio, win=512, step=256):
    '''
    Compute the spectrogram of audio data

    audio: one channel audio
    win: window size for dft sliding window
    step: step size for dft sliding windo
    '''
    spectrogram = []
    hanning = np.hanning(win)
    for i in range(win, len(audio), step):
        start = win // 2
        dft = np.abs(fft(audio[i - win: i] * hanning))[start:win]
        spectrogram.append(dft)
    return np.array(spectrogram)

In [3]:
# PASSWORDS AND STUFF HERE

In [4]:
settings = {
   'user': db_user,
   'pass': db_password,
   'host': host,
     'db': db_name
}
url = 'mysql+pymysql://{user}:{pass}@{host}/{db}'.format(**settings)  # 5432 is the default port
db = sqlalchemy.create_engine(url)

In [5]:
def run_query(query):
    with db.connect() as conn:
        rows = []
        for row in conn.execute(query).fetchall():
            rows.append(dict(row.items()))
        return rows

In [6]:
files = run_query("""
    SELECT 
        x.encoding, y.year, x.filename 
    FROM 
        wdp_ds.audio x 
    JOIN wdp_ds.encoding y ON x.encoding = y.encoding;
""")

In [7]:
paths = ["audio_files/{}/{}".format(file['year'], file['filename']) for file in files]

In [8]:
max_id = run_query("""
    SELECT 
        max(id)
    FROM 
        wdp_ds.not_silent
    """)
if max_id[0]['max(id)'] is None:
    max_id = 0
else:
    max_id = max_id[0]['max(id)']
print(max_id)

0


In [9]:
noise_classifier = load_model('../models/lstm_v2/sil.h5')
client = storage.Client.from_service_account_json('../secret.json')
bucket = client.get_bucket('wdp-data')

In [ ]:
id = max_id + 1
for path, file_dict in zip(paths, files):
    print(path, file_dict)
    with open("/tmp/audio.m4a", "wb") as file_obj:
        blob = bucket.blob(path)
        blob.download_to_file(file_obj)
    stream = StreamSpectrogram("/tmp/audio.m4a")
    not_noise = []
    for x in stream:
        y = noise_classifier.predict(x).flatten()
        not_noise.extend([int(np.round(sample)) == 0 for sample in y])

    regions = []
    for i in range(0, len(not_noise)):
        if not_noise[i]:
            start = i * 16 * 256 
            stop  = (i + 1) * 16 * 256 
            if len(regions) > 0: 
                last  = regions[-1]
                if start - last[1] < 48000 * 0.1:
                    start       = regions[-1][0]
                    regions[-1] = (start, stop)
                else:
                    regions.append((start, stop))
            else:
                regions.append((start, stop))
    regions = [(start, stop) for start, stop in regions if stop - start > (16 * 256)]
    if len(regions) > 0:
        with db.connect() as conn:
            for start, stop in regions:                
                conn.execute("INSERT INTO wdp_ds.not_silent VALUES ({}, {}, '{}', {}, {})".format(id, file_dict['encoding'], file_dict['filename'], start, stop))  
                id += 1
    print(file_dict['encoding'], ": ", regions)

5281102 :  [(471040, 479232), (724992, 745472), (778240, 794624), (946176, 954368), (991232, 999424), (1007616, 1060864), (1777664, 1785856), (1908736, 1933312), (2023424, 2072576), (2170880, 2183168), (2195456, 2203648), (2600960, 2613248), (5541888, 5554176), (5623808, 5632000), (5640192, 5664768), (7532544, 7544832), (7774208, 7782400), (8290304, 8298496), (9416704, 9469952), (9601024, 9650176), (11259904, 11268096), (11288576, 11296768), (11395072, 11407360), (14008320, 14016512), (14024704, 14036992), (14159872, 14176256), (17256448, 17276928), (17608704, 17620992), (17629184, 17637376), (17694720, 17702912), (17711104, 17719296), (18059264, 18075648), (18931712, 18939904), (19632128, 19640320), (20795392, 20807680), (20987904, 21000192), (21803008, 21815296), (22839296, 22851584), (24543232, 24551424), (24694784, 24702976), (24846336, 24854528), (24915968, 24928256), (25186304, 25223168), (25296896, 25309184), (25591808, 25620480), (25632768, 25657344), (25665536, 25673728), (269